In [1]:
import pickle
import numpy as np
# import pandas as pd
import os
from openai import OpenAI
import httpx

def save_pickle(obj, filename):
    _, ext = os.path.splitext(filename)
    if ext in ['.pkl','.p','.data']:
        with open(filename, "wb") as f:
            pickle.dump(obj, f)
    elif ext == '.npy':
        if not isinstance(obj, np.ndarray):
            obj = np.array(obj)
        np.save(filename, obj)
    else:
        pass # raise Error

def load_pickle(filename):
    _, ext = os.path.splitext(filename)
    if ext in ['.pkl','.p','.data']:
        with open(filename, "rb") as f:
            data = pickle.load(f)
        return data
    elif ext == '.npy':
        return np.load(filename)
    else:
        return None # raise Error


In [2]:
# read from down sampling results
# apply sudo label assignment

import regex
import json

# down_sampling_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/input_sample.jsonl"
# down_sampling_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/generation_1/generation_1.jsonl"
# down_sampling_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1/generation_1.jsonl"
down_sampling_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Weibo-Aminer/topic_llm2/generation_1/generation_1.jsonl"

# regex_pattern = r"\[\d+\] ([\w\s]+)"
regex_pattern = r"\[\d+\] ([\u4e00-\u9fff]+)"

def read_from_topicgpt(filepath):
    user_tweets = {}
    with open(filepath, "r") as f:
        for line in f:
            info = json.loads(line)
            text = info["text"]
            mt_info = info["meta_info"]
            topic = info["responses"]
            topic = regex.compile(regex_pattern).findall(topic)
            if len(topic) > 0: topic = topic[0]
            else: topic = "None"
            if mt_info["user"] not in user_tweets: user_tweets[mt_info["user"]] = []
            user_tweets[mt_info["user"]].append({
                "user": mt_info["user"],
                "ts": mt_info["ts"],
                "text": text,
                "tag": mt_info["tag"],
                "bertopic_label": mt_info["bertopic_label"],
                "topicgpt_label": topic,
            })
    return user_tweets

user_infos = read_from_topicgpt(down_sampling_filepath)
print(len(user_infos))
print(sum([len(tweets) for user, tweets in user_infos.items()]))

# for user, infos in user_infos.items():
#     print(user)
#     for info in infos:
#         print(info)
#         break
#     break

1501
14000


In [4]:
# # Get other unlabeled data
# cascade_dict = load_pickle("/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/cascades.data")

# user_tweets_dict = {}
# for tag, cascades in cascade_dict.items():
#     for user, ts, tweet, bertopic_label in zip(cascades['user'], cascades['ts'], cascades['content'], cascades['label']):
#         if user not in user_tweets_dict:
#             user_tweets_dict[user] = []
#         user_tweets_dict[user].append({
#             "tag": tag,
#             "bertopic_label": str(bertopic_label),
#             "user": user,
#             "ts": ts,
#             "text": tweet,
#         })
# print(sum([len(tweets) for user, tweets in user_tweets_dict.items()]))

# user_remain_infos = {}
# user_seeds = user_infos.keys()
# for user, full_infos in user_tweets_dict.items():
#     if user not in user_seeds:
#         # user_remain_infos[user] = full_infos
#         continue

#     infos = user_infos[user]
#     remain_infos = []
#     for info in full_infos:
#         flag = False
#         for s_info in infos:
#             if info["text"] == s_info["text"] \
#                 and info["tag"] == s_info["tag"] \
#                 and info["user"] == s_info["user"] \
#                 and info["ts"] == s_info["ts"]:
#                 flag = True
#                 break
#         if not flag:
#             remain_infos.append(info)
#     # print(len(infos), len(remain_infos), len(full_infos), "{:.2f}".format(len(remain_infos) / len(full_infos)))
#     # try:
#     #     assert len(infos) + len(remain_infos) == len(full_infos)
#     # except Exception as e:
#     #     print(user, len(infos), len(remain_infos), len(full_infos))
#     user_remain_infos[user] = remain_infos

# print(sum([len(infos) for infos in user_remain_infos.values()]))

568256
106988


In [3]:
ch_to_en = {
    "科技": "Technology",
    "体育": "Sports",
    "社会问题": "Social Issues",
    "历史": "History",
    "娱乐": "Entertainment",
    "农业": "Agriculture",
    "政治": "Politics",
    "军事": "Military",
    "教育": "Education",
    "经济": "Economy",
    "环境": "Environment",
    "贸易": "Trade",
    "文化": "Culture",
    "地缘政治": "Geopolitics",
    "文学": "Literature",
    "食品": "Food",
    "天气": "Weather",
    "媒体": "Media",
    "社交媒体": "Social Media",
    "外交": "Diplomacy",
    "犯罪": "Crime",
    "安全": "Security",
    "健康": "Health",
    "交通": "Transportation",
    "政府": "Government",
    "国际关系": "International Relations",
    "法律": "Law",
    "宗教": "Religion",
    "旅游": "Tourism",
    "食品安全": "Food Safety",
    "商业": "Business",
    "社交": "Social Interaction"
}

en_to_ch = {v: k for k, v in ch_to_en.items()}

In [4]:
# 2. Use trained bertopic model to assign sudo labels

from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, util

# TODO: use broader interests
# gt_ft_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/gt_ft_mapping.data"
gt_ft_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Weibo-Aminer/topic_llm2/gt_ft_mapping.data"
gt_ft_mapping = load_pickle(gt_ft_filepath)

# Get labeled data
data = []
target = []
for user, infos in user_infos.items():
    for info in infos:
        if info["topicgpt_label"] == "None": continue
        if info["topicgpt_label"] not in ch_to_en: continue
        if ch_to_en[info["topicgpt_label"]] not in gt_ft_mapping: continue
        data.append(info["text"])
        # target.append(info["topicgpt_label"])
        target.append(gt_ft_mapping[ch_to_en[info["topicgpt_label"]]])

# Discretize targets
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# target = label_encoder.fit_transform(target)

print(len(data))
print(sum([len(tweets) for user, tweets in user_infos.items()]))

# split into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)


/root/anaconda3/envs/topicGPT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


6220
14000


In [ ]:
# # 1. Bertopic Model

# # Skip over dimensionality reduction, replace cluster model with classifier,
# # and reduce frequent words while we are at it.
# empty_dimensionality_model = BaseDimensionalityReduction()
# clf = LogisticRegression()
# ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
# sbert = SentenceTransformer("/remote-home/share/dmb_nas/wangzejian/LLM_GNN/all-MiniLM-L6-v2")

# # Create a fully supervised BERTopic instance
# topic_model= BERTopic(
#     umap_model=empty_dimensionality_model,
#     hdbscan_model=clf,
#     ctfidf_model=ctfidf_model,
#     embedding_model=sbert,
# )
# topic_model = topic_model.fit(X_train, y=y_train)

# # Map input `y` to topics
# mappings = topic_model.topic_mapper_.get_mappings()
# # reverse_mapping = {v: k for k, v in mappings.items()}
# # print(mappings)

# y_preds = []
# topics, _ = topic_model.transform(X_test[:100])
# # interest_labels = label_encoder.inverse_transform([reverse_mapping[topic] for topic in topics])
# interest_labels = [mappings[topic] for topic in topics]
# print(interest_labels)
# # y_preds.append(mappings[topic[0]])

# # test accuracy
# from sklearn.metrics import accuracy_score
# print(accuracy_score(y_test[:100], interest_labels))

In [5]:
# 2. Direct SBert Cosine Similarity

# Get embeddings
sbert = SentenceTransformer("/remote-home/share/dmb_nas/wangzejian/LLM_GNN/all-MiniLM-L6-v2")
embeddings = sbert.encode(X_train)

topic_embeddings = {}
for emb, label in zip(embeddings, y_train):
    if label not in topic_embeddings:
        topic_embeddings[label] = []
    topic_embeddings[label].append(emb)

for interest, emb in topic_embeddings.items():
    topic_embeddings[interest] = np.mean(emb, axis=0)
    # print(topic_embeddings[interest].shape)
key2idx = {i: k for i, k in enumerate(topic_embeddings.keys())}
topic_t_embeddings = np.array([emb for emb in topic_embeddings.values()])

# Get test embeddings
test_embeddings = sbert.encode(X_test[:100])
# print(topic_t_embeddings.shape)
# print(test_embeddings.shape)

# Get cosine similarity and Give Labels
cosine_scores = util.pytorch_cos_sim(test_embeddings, topic_t_embeddings)
# print(cosine_scores.shape)
predictions = np.argmax(cosine_scores, axis=1).tolist()
predictions = [key2idx[p] for p in predictions]
# print(predictions)
# print(y_test[:100])

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test[:100], predictions))


0.14


In [16]:
# remain_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Weibo-Aminer/topic_llm2/input_sample_remain.jsonl"
# user_remains_info = {}
# with open(remain_filepath, 'r') as f:
#     for line in f:
#         info = json.loads(line)
#         user = info["meta_info"]["user"]
#         if user not in user_remains_info:
#             user_remains_info[user] = []
#         user_remains_info[user].append(info)

# print(sum([len(infos) for infos in user_remains_info.values()]))

63932


In [8]:
import random

# Get other unlabeled data
cascade_dict = load_pickle("/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/cascades.data")

user_tweets_dict = {}
for tag, cascades in cascade_dict.items():
    for user, ts, tweet, bertopic_label in zip(cascades['user'], cascades['ts'], cascades['content'], cascades['label']):
        if user not in user_tweets_dict:
            user_tweets_dict[user] = []
        user_tweets_dict[user].append({
            "tag": tag,
            "bertopic_label": str(bertopic_label),
            "user": user,
            "ts": ts,
            "text": tweet,
        })
print(sum([len(tweets) for user, tweets in user_tweets_dict.items()]))

candidate_users = list(
    set(list(user_tweets_dict.keys())) - \
        set(list(user_infos.keys())))
expanded_users = random.sample(candidate_users, k=2000)


# Get Remaining Info
remain_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Weibo-Aminer/topic_llm2/input_sample_remain.jsonl"
user_remains_info = {}
with open(remain_filepath, 'r') as f:
    for line in f:
        info = json.loads(line)
        user = info["meta_info"]["user"]
        if user not in user_remains_info:
            user_remains_info[user] = []
        user_remains_info[user].append(info)

print(sum([len(infos) for infos in user_remains_info.values()]))

# 1. Sample only 2000 / full users to expand
for user in expanded_users:
    infos = user_tweets_dict[user]
    user_remains_info[user] = [
        {
            "meta_info": {
                "tag": info["tag"],
                "user": info["user"],
                "ts": info["ts"],
                "bertopic_label": info["bertopic_label"],
            },
            "text": info["text"],
            # "topicgpt_label": "None",
        }
        for info in infos
    ]

print(sum([len(infos) for infos in user_remains_info.values()]))

# 2. Sample At most 20 tweets for each user
# for user in candidate_users:
#     user_remains_info[user] = random.sample(list(user_tweets_dict[user]), k=min(len(user_tweets_dict[user]), 20))

# t_len = []
# for user, infos in user_remains_info.items():
#     t_len.append(len(infos))

# def analyse_distribution(data):
#     for i in range(10):
#         t = np.percentile(data, i*10)
#         print(t)
# analyse_distribution(t_len)

# for user in expanded_users:
#     user_remains_info[user] = user_tweets_dict[user]

568256
63932
174757


In [14]:
# # Get other unlabeled data

# user_remain_infos = {}
# user_seeds = user_infos.keys()
# for user, full_infos in user_tweets_dict.items():
#     if user not in user_seeds:
#         if user in expanded_users:
#             user_remain_infos[user] = full_infos
#         continue

#     infos = user_infos[user]
#     remain_infos = []
#     for info in full_infos:
#         flag = False
#         for s_info in infos:
#             if info["text"] == s_info["text"] \
#                 and info["tag"] == s_info["tag"] \
#                 and info["user"] == s_info["user"] \
#                 and info["ts"] == s_info["ts"]:
#                 flag = True
#                 break
#         if not flag:
#             remain_infos.append(info)
#     # print(len(infos), len(remain_infos), len(full_infos), "{:.2f}".format(len(remain_infos) / len(full_infos)))
#     # try:
#     #     assert len(infos) + len(remain_infos) == len(full_infos)
#     # except Exception as e:
#     #     print(user, len(infos), len(remain_infos), len(full_infos))
#     user_remain_infos[user] = remain_infos

# print(sum([len(infos) for infos in user_remain_infos.values()]))

127169


In [10]:
# user_remain_infos_cp = {}

# save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1.jsonl"
filename = "ex_2000user"
save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Weibo-Aminer/topic_llm2/generation_1/generation_1_{}.jsonl".format(filename)

with open(save_filepath, 'w') as f:
    # for user, infos in user_remain_infos.items():
    for user, infos in user_remains_info.items():
        testing_texts = [info["text"] for info in infos]
        test_embeddings = sbert.encode(X_test[:100])
        cosine_scores = util.pytorch_cos_sim(test_embeddings, topic_t_embeddings)
        # print(cosine_scores.shape)
        predictions = np.argmax(cosine_scores, axis=1).tolist()
        predictions = [key2idx[p] for p in predictions]

        # user_remain_infos_cp[user] = []
        for info, pred in zip(infos, predictions):
            info["topicgpt_label"] = pred
            f.write(json.dumps(info) + "\n")
            # user_remain_infos_cp[user].append(info)


In [24]:
idx = 0
for ypred, ytrue in zip(predictions, y_test):
    if idx > 100: break
    if ypred != ytrue:
        print(X_test[idx])
        print("---" + ypred + "---" + ytrue + "---")
        # print(label_encoder.inverse_transform([ypred]), label_encoder.inverse_transform([ytrue]))
    idx += 1

RT @HananyaNaftali: The Church of the Holy Sepulchre in #Jerusalem is closed due to Coronavirus. #Israel

The last time it was closed was i…
---Health---Religion---
RT @krislc: #政治攬炒 political mutual destruction has begun. Question is: Beijing knows this will likely happen when deciding to break from tr…
---Military---Politics---
Pakistan condemned the inhuman Syrian Regime attack on Turkish troops. Pakistanis are with the Turks at this critical time; #sehitlerimizvar
---Academic_disciplines---Military---
RT @www_HKer: For 🇹🇭Thai friends: 

#Rule1 to fight #CCPViurs is : you do not trust CCP
#Rule2 : you DO NOT trust CCP
#Rule3 : remember the…
---Politics---Military---
RT @Yfreeeeeeehk: Lovely #HongKongers❤️
Forwadly to clean up the Mosque after police defaced it by a water cannon.
#PoliceBrutalitiy #HongK…
---Society---Food_and_drink---
#Gambian newspapers #DailyNews and @PointGambia highlight today #4Mar, in its printed editions, the solidarity voca… https://t.co/u1R13oWLB4
---Academ